In [ ]:
import polars as pl
from datetime import datetime, date
import hvplot.polars

In [ ]:
# Load datasets
un = pl.read_csv("data/un_basic.csv", try_parse_dates=True)
forest_area = pl.read_csv("data/our_world_in_data/forest-area-km.csv")
weather = pl.read_parquet("data/vienna-meteostat.parquet")

In [ ]:
# Solution load-cities
cities = pl.read_parquet("data/worldcities.parquet")
cities

In [ ]:
# Solution world-map
cities.hvplot.scatter(
    x="lng",
    y="lat",
    # The following arguments are optional
    hover_cols=["city"],
    color="country",
    title="Cities of the World",
    height=500,
    width=1000,
    legend=False,
    grid=True
)

In [ ]:
# Solution ten-smallest
ten_smallest = pl.sql("SELECT * FROM un ORDER BY population LIMIT 10").collect()
ten_smallest.hvplot.bar(x="iso3", y="population", color="region", hover_cols=["country"])

In [ ]:
# Solution membership-years
current_date = date.today()
pl.sql(f"SELECT *, ('{current_date}'::DATE - admission_date)::INTERVAL AS membership_in_years FROM un").collect()
# TODO: How to convert the years to days (INTERVAL '1 year' does not work)


In [ ]:
# Solution energy-at
el_source = pl.read_csv("data/our_world_in_data/electricity-source.csv", infer_schema_length=5000)
el_source_austria = pl.sql("SELECT * FROM el_source WHERE country = 'Austria'").collect()
el_source_austria

In [ ]:
# Solution energy-at (1)
el_source_austria.hvplot.area(x="year", y=["nuclear", "hydro", "fossil", "renewables"], stacked=True)

In [ ]:
# Solution energy-at (2)
el_source_austria.hvplot(x="year", y=["nuclear", "hydro", "fossil", "renewables"])

In [ ]:
# Solution founding-members
first_date = un["admission_date"].min()
founding_members = pl.sql(f"SELECT * FROM un WHERE admission_date == '{first_date}'::DATE").collect()
founding_members

In [ ]:
# Solution forest-change
pl.sql("""
    SELECT Entity, (area_last - area_first) / area_first AS rel_diff
    FROM
    (
        SELECT Entity, FIRST("Forest area") AS area_first, LAST("Forest area") AS area_last 
        FROM forest_area
        GROUP BY Entity
    ) AS f
    WHERE area_first > 0 AND area_last > 0 
    ORDER BY rel_diff
""").collect()

In [ ]:
# Solution hottest-night
pl.sql("""
    SELECT time::DATE AS date, min(temp) as min_temp FROM weather
    WHERE time > '1980-01-01'
    GROUP BY time::DATE
    ORDER BY min_temp DESC
    LIMIT 10
"""
).collect()

In [ ]:
# Solution million-cities (bonus)
million_cities_per_region = pl.sql("""
    SELECT region, subregion, count(*) AS count, first(city), first(population) FROM
    (
        SELECT * FROM cities ORDER by population DESC
    ) AS c
    INNER JOIN un on c.iso3 = un.iso3
    WHERE population > 1000000
    GROUP BY region, subregion
    ORDER BY count DESC
""").collect()
million_cities_per_region


In [ ]:
# Solution forest-region
# TODO: I don't know yet how to do it
forest_area_by_region = forest_area.join(un, left_on="Code", right_on="iso3", how="inner").pivot(
    on="region", values="Forest area", index="Year", aggregate_function="sum"
)
forest_area_by_region.hvplot.area(stacked=True, x="Year")